# EP 3: Machine Learning 101

📚 สนใจเรียน/ปรึกษา(Fastwork): <a href="https://fastwork.co/user/krittachailouis/tutoring-12693427">https://fastwork.co/user/krittachailouis/tutoring-12693427</a>

<img width="32" alt="YouTube icon" src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/09/YouTube_full-color_icon_%282017%29.svg/32px-YouTube_full-color_icon_%282017%29.svg.png?20240107144800"> Youtube:
 <a href="https://commons.wikimedia.org/wiki/File:YouTube_full-color_icon_(2017).svg">
  Louis MakerLab
 </a>
 
<img width="32" alt="GitHub icon" src="https://github.githubassets.com/images/modules/logos_page/GitHub-Mark.png"> GitHub: 
<a href="https://github.com/louisa9555">
  Louis MakerLab
</a>


## Linear Models, Tree-based Models และ Kernel Methods
### เนื้อหาในบทเรียนนี้:
1. **Linear Models (โมเดลเชิงเส้น)**
   - Linear Regression (การถดถอยเชิงเส้น)
   - Logistic Regression (การถดถอยโลจิสติก)
2. **Tree-based Models (โมเดลที่ใช้ต้นไม้)**
   - Decision Trees (ต้นไม้ตัดสินใจ)
   - Random Forests (ป่าสุ่ม)
   - XGBoost
3. **Kernel Methods (วิธีเคอร์เนล)**
   - Support Vector Machine (SVM)
   - Gaussian Processes

---

### วัตถุประสงค์การเรียนรู้:
- เข้าใจหลักการทำงานของโมเดลแต่ละประเภท
- สามารถเลือกโมเดลที่เหมาะสมกับปัญหาที่กำหนด
- ใช้โค้ด Python เพื่อสร้างและประเมินโมเดล
- เปรียบเทียบประสิทธิภาพของโมเดลต่างๆ

## การติดตั้ง
- สร้าง conda env
- `conda install -c conda-forge llvm-openmp`
- `brew install libomp`
- pip install numpy pandas matplotlib seaborn scikit-learn xgboost

In [ ]:
# นำเข้าไลบรารีที่จำเป็น
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification, make_regression, load_iris, load_breast_cancer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, mean_squared_error, classification_report, confusion_matrix

# Linear Models
from sklearn.linear_model import LinearRegression, LogisticRegression

# Tree-based Models
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
import xgboost as xgb

# Kernel Methods
from sklearn.svm import SVC, SVR
from sklearn.gaussian_process import GaussianProcessClassifier, GaussianProcessRegressor

# การตั้งค่า matplotlib สำหรับการแสดงผลภาษาไทย
import matplotlib.font_manager as fm
plt.rcParams['figure.figsize'] = (10, 6)
sns.set_style("whitegrid")

print("นำเข้าไลบรารีเรียบร้อยแล้ว!")

# ฟังก์ชันสำหรับตั้งค่าฟอนต์ไทยใน matplotlib
def setup_matplotlib_thai_font():
    """Find and return a Thai font if available"""
    # Candidate font names that support Thai
    thai_fonts = ["TH Sarabun New", "Noto Sans Thai", "Noto Serif Thai", "Tahoma", "Angsana New"]

    # Get list of installed fonts
    font_list = fm.findSystemFonts(fontpaths=None, fontext='ttf')
    for font_path in font_list:
        prop = fm.FontProperties(fname=font_path)
        try:
            name = prop.get_name()
            if any(tf in name for tf in thai_fonts):
                print(f"✔ Using Thai font: {name}")
                return prop
        except Exception:
            pass
    print("⚠️  ไม่พบฟอนต์ไทย, ใช้ฟอนต์เริ่มต้น.")
    return None

# ตั้งค่าฟอนต์ matplotlib
matplotlib_thai_font = setup_matplotlib_thai_font()

## ส่วนที่ 2: Tree-based Models (โมเดลที่ใช้ต้นไม้)

### 2.1 Decision Trees (ต้นไม้ตัดสินใจ)

**ทฤษฎี:**
- Decision Trees สร้างกฎการตัดสินใจในรูปของต้นไม้
- แต่ละ node ของต้นไม้เป็นเงื่อนไขในการแบ่งข้อมูล
- ใช้เกณฑ์เช่น Gini Impurity หรือ Information Gain ในการแบ่ง

**จุดแข็ง:**
- ง่ายต่อการตีความและอธิบาย
- ไม่ต้องการการปรับ scaling ข้อมูล
- สามารถจัดการกับ features ที่เป็นตัวเลขและตัวหนังสือได้

**จุดอ่อน:**
- มีแนวโน้ม overfitting
- ไม่เสถียร (เปลี่ยนข้อมูลเล็กน้อยอาจทำให้โมเดลเปลี่ยนมาก)
- อาจไม่ดีกับข้อมูลเชิงเส้น

In [ ]:
# ใช้ข้อมูล Iris dataset สำหรับ Decision Tree
from sklearn.datasets import load_iris
iris = load_iris()
X_iris, y_iris = iris.data, iris.target
print(X_iris.shape)
print("(0 = Setosa, 1 = Versicolor, 2 = Virginica)")
print(y_iris)
# plot ข้อมูลเบื้องต้น
df_iris = pd.DataFrame(X_iris, columns=iris.feature_names)
df_iris['target'] = y_iris
df_iris['target'] = df_iris['target'].map({0: 'Setosa', 1: 'Versicolor', 2: 'Virginica'})
display(df_iris.head())
if matplotlib_thai_font:
	plt.rcParams['font.family'] = matplotlib_thai_font.get_name()
sns.pairplot(df_iris, hue='target', diag_kind='kde')
plt.suptitle('ข้อมูล Iris Dataset', y=1.02)
plt.show()


In [ ]:
# แบ่งข้อมูล
X_train_tree, X_test_tree, y_train_tree, y_test_tree = train_test_split(
    X_iris, y_iris, test_size=0.3
)

# สร้างโมเดล Decision Tree
dt_classifier = DecisionTreeClassifier(max_depth=3)
dt_classifier.fit(X_train_tree, y_train_tree)

# ทำนายผล
y_pred_tree = dt_classifier.predict(X_test_tree)

# คำนวณความแม่นยำ
accuracy_tree = accuracy_score(y_test_tree, y_pred_tree)
print(f"ความแม่นยำของ Decision Tree: {accuracy_tree:.3f}")

# แสดง Feature Importance
feature_importance = dt_classifier.feature_importances_
print("\nFeature Importance:", feature_importance)
feature_names = iris.feature_names

print("\nความสำคัญของ Features:")
for i, (name, importance) in enumerate(zip(feature_names, feature_importance)):
    print(f"{name}: {importance:.3f}")

# แสดงกราฟ Feature Importance
plt.figure(figsize=(10, 6))
plt.bar(range(len(feature_importance)), feature_importance)
plt.xticks(range(len(feature_names)), feature_names, rotation=45)
plt.title('Feature Importance ใน Decision Tree')
plt.ylabel('Importance')
plt.tight_layout()
plt.show()

# แสดง Confusion Matrix
plt.figure(figsize=(8, 6))
cm_tree = confusion_matrix(y_test_tree, y_pred_tree)
sns.heatmap(cm_tree, annot=True, fmt='d', cmap='Blues', 
            xticklabels=iris.target_names, yticklabels=iris.target_names)
plt.title('Confusion Matrix - Decision Tree')
plt.xlabel('ค่าทำนาย')
plt.ylabel('ค่าจริง')
plt.show()

### 2.2 Random Forests (ป่าสุ่ม, Ensemble learning technique)

**ทฤษฎี:**
- Random Forest เป็น ensemble method ที่รวม Decision Trees หลายๆ ต้น
- ใช้ bagging (bootstrap aggregating) และ random feature selection
- การทำนายขั้นสุดท้ายมาจากการโหวตหรือเฉลี่ยของต้นไม้ทั้งหมด

**จุดแข็ง:**
- ลด overfitting ได้ดีกว่า Decision Tree เดี่ยว
- ให้ประสิทธิภาพที่ดีในหลายปัญหา
- สามารถจัดการกับข้อมูลที่มี noise ได้ดี

**จุดอ่อน:**
- ตีความยากกว่า Decision Tree เดี่ยว
- ใช้หน่วยความจำมากกว่า
- อาจไม่เหมาะกับข้อมูลเชิงเส้นง่ายๆ

In [ ]:
# สร้างโมเดล Random Forest
rf_classifier = RandomForestClassifier(n_estimators=100, max_depth=3)
rf_classifier.fit(X_train_tree, y_train_tree)

# ทำนายผล
y_pred_rf = rf_classifier.predict(X_test_tree)

# คำนวณความแม่นยำ
accuracy_rf = accuracy_score(y_test_tree, y_pred_rf)
print(f"ความแม่นยำของ Random Forest: {accuracy_rf:.3f}")

# เปรียบเทียบกับ Decision Tree
print(f"ความแม่นยำของ Decision Tree: {accuracy_tree:.3f}")
print(f"การปรับปรุง: {accuracy_rf - accuracy_tree:.3f}")

# แสดง Feature Importance สำหรับ Random Forest
rf_feature_importance = rf_classifier.feature_importances_

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.bar(range(len(feature_importance)), feature_importance)
plt.xticks(range(len(feature_names)), feature_names, rotation=45)
plt.title('Feature Importance - Decision Tree')
plt.ylabel('Importance')

plt.subplot(1, 2, 2)
plt.bar(range(len(rf_feature_importance)), rf_feature_importance)
plt.xticks(range(len(feature_names)), feature_names, rotation=45)
plt.title('Feature Importance - Random Forest')
plt.ylabel('Importance')

plt.tight_layout()
plt.show()

# Cross-validation เพื่อเปรียบเทียบประสิทธิภาพ
dt_scores = cross_val_score(dt_classifier, X_iris, y_iris, cv=5)
rf_scores = cross_val_score(rf_classifier, X_iris, y_iris, cv=5)

print(f"\nCross-validation scores:")
print(f"Decision Tree: {dt_scores.mean():.3f} ± {dt_scores.std():.3f}")
print(f"Random Forest: {rf_scores.mean():.3f} ± {rf_scores.std():.3f}")

### 2.3 XGBoost (Extreme Gradient Boosting, ภาคต่อจาก Decision Tree)

**ทฤษฎี:**
- XGBoost เป็น gradient boosting algorithm ที่มีประสิทธิภาพสูง
- สร้างโมเดลใหม่เพื่อแก้ไขข้อผิดพลาดของโมเดลเก่า
- ใช้ regularization เพื่อป้องกัน overfitting

**จุดแข็ง:**
- ประสิทธิภาพสูงมากในการแข่งขัน machine learning
- มี built-in regularization
- สามารถจัดการกับ missing values ได้

**จุดอ่อน:**
- ซับซ้อนในการปรับ hyperparameters
- ใช้เวลาในการฝึกนานกว่า Random Forest
- ต้องการความเข้าใจลึกในการปรับแต่ง

In [ ]:
# สร้างโมเดล XGBoost
xgb_classifier = xgb.XGBClassifier(max_depth=3, n_estimators=100, eta=0.3)
xgb_classifier.fit(X_train_tree, y_train_tree)

# ทำนายผล
y_pred_xgb = xgb_classifier.predict(X_test_tree)

# คำนวณความแม่นยำ
accuracy_xgb = accuracy_score(y_test_tree, y_pred_xgb)
print(f"ความแม่นยำของ XGBoost: {accuracy_xgb:.3f}")

# เปรียบเทียบกับโมเดลอื่น
print("\nเปรียบเทียบความแม่นยำ:")
print(f"Decision Tree: {accuracy_tree:.3f}")
print(f"Random Forest: {accuracy_rf:.3f}")
print(f"XGBoost: {accuracy_xgb:.3f}")

# แสดง Feature Importance สำหรับ XGBoost
xgb_feature_importance = xgb_classifier.feature_importances_

plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.bar(range(len(feature_importance)), feature_importance)
plt.xticks(range(len(feature_names)), feature_names, rotation=45)
plt.title('Decision Tree')
plt.ylabel('Importance')

plt.subplot(1, 3, 2)
plt.bar(range(len(rf_feature_importance)), rf_feature_importance)
plt.xticks(range(len(feature_names)), feature_names, rotation=45)
plt.title('Random Forest')
plt.ylabel('Importance')

plt.subplot(1, 3, 3)
plt.bar(range(len(xgb_feature_importance)), xgb_feature_importance)
plt.xticks(range(len(feature_names)), feature_names, rotation=45)
plt.title('XGBoost')
plt.ylabel('Importance')

plt.suptitle('เปรียบเทียบ Feature Importance ของโมเดลต่างๆ')
plt.tight_layout()
plt.show()

# Cross-validation
xgb_scores = cross_val_score(xgb_classifier, X_iris, y_iris, cv=5)
print(f"\nCross-validation scores:")
print(f"Decision Tree: {dt_scores.mean():.3f} ± {dt_scores.std():.3f}")
print(f"Random Forest: {rf_scores.mean():.3f} ± {rf_scores.std():.3f}")
print(f"XGBoost: {xgb_scores.mean():.3f} ± {xgb_scores.std():.3f}")